# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 2 2022 8:43AM,243236,10,0085919853,ISDIN Corporation,Released
1,Aug 2 2022 8:43AM,243236,10,0085919876,ISDIN Corporation,Released
2,Aug 2 2022 8:43AM,243236,10,0085919883,ISDIN Corporation,Released
3,Aug 2 2022 8:43AM,243236,10,0085919906,ISDIN Corporation,Released
4,Aug 2 2022 8:43AM,243236,10,0085919930,ISDIN Corporation,Released
5,Aug 2 2022 8:43AM,243236,10,0085919933,ISDIN Corporation,Released
6,Aug 2 2022 8:43AM,243236,10,0085919939,ISDIN Corporation,Released
7,Aug 2 2022 8:43AM,243236,10,0085921253,ISDIN Corporation,Released
8,Aug 2 2022 8:37AM,243156,20,8346351,"ACI Healthcare USA, Inc.",Released
9,Aug 2 2022 8:37AM,243156,20,8346341,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,243232,Released,1
33,243233,Released,4
34,243234,Released,4
35,243235,Released,1
36,243236,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243232,NaN,NaN,1.0
243233,NaN,NaN,4.0
243234,NaN,NaN,4.0
243235,NaN,NaN,1.0
243236,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0.0,0.0,1.0
243129,0.0,0.0,1.0
243145,21.0,23.0,3.0
243152,0.0,1.0,1.0
243156,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0,0,1
243129,0,0,1
243145,21,23,3
243152,0,1,1
243156,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,0,0,1
1,243129,0,0,1
2,243145,21,23,3
3,243152,0,1,1
4,243156,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,,,1
1,243129,,,1
2,243145,21,23,3
3,243152,,1,1
4,243156,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation
8,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc."
11,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated"
14,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc."
15,Aug 2 2022 8:30AM,243202,10,Emerginnova
17,Aug 2 2022 8:30AM,243215,10,Eywa Pharma Inc.
21,Aug 2 2022 8:30AM,243203,10,"Methapharm, Inc."
22,Aug 2 2022 8:30AM,243219,10,"Methapharm, Inc."
26,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation
28,Aug 2 2022 8:30AM,243213,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,,,8
1,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc.",,,3
2,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated",,,3
3,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc.",,,1
4,Aug 2 2022 8:30AM,243202,10,Emerginnova,,,2
5,Aug 2 2022 8:30AM,243215,10,Eywa Pharma Inc.,,,4
6,Aug 2 2022 8:30AM,243203,10,"Methapharm, Inc.",,,1
7,Aug 2 2022 8:30AM,243219,10,"Methapharm, Inc.",,,4
8,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation,,,6
9,Aug 2 2022 8:30AM,243213,10,ISDIN Corporation,,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,8,,
1,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc.",3,,
2,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated",3,,
3,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc.",1,,
4,Aug 2 2022 8:30AM,243202,10,Emerginnova,2,,
5,Aug 2 2022 8:30AM,243215,10,Eywa Pharma Inc.,4,,
6,Aug 2 2022 8:30AM,243203,10,"Methapharm, Inc.",1,,
7,Aug 2 2022 8:30AM,243219,10,"Methapharm, Inc.",4,,
8,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation,6,,
9,Aug 2 2022 8:30AM,243213,10,ISDIN Corporation,13,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,8,,
1,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc.",3,,
2,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated",3,,
3,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc.",1,,
4,Aug 2 2022 8:30AM,243202,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,8,NaN,NaN
1,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc.",3,NaN,NaN
2,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated",3,NaN,NaN
3,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc.",1,NaN,NaN
4,Aug 2 2022 8:30AM,243202,10,Emerginnova,2,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,8,0.0,0.0
1,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc.",3,0.0,0.0
2,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated",3,0.0,0.0
3,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc.",1,0.0,0.0
4,Aug 2 2022 8:30AM,243202,10,Emerginnova,2,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4377883,65,0.0,0.0
16,243233,4,0.0,0.0
18,243232,1,0.0,0.0
19,1459128,25,2.0,0.0
20,972556,25,50.0,21.0
21,243218,1,0.0,0.0
22,243214,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4377883,65,0.0,0.0
1,16,243233,4,0.0,0.0
2,18,243232,1,0.0,0.0
3,19,1459128,25,2.0,0.0
4,20,972556,25,50.0,21.0
5,21,243218,1,0.0,0.0
6,22,243214,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,65,0.0,0.0
1,16,4,0.0,0.0
2,18,1,0.0,0.0
3,19,25,2.0,0.0
4,20,25,50.0,21.0
5,21,1,0.0,0.0
6,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,65.0
1,16,Released,4.0
2,18,Released,1.0
3,19,Released,25.0
4,20,Released,25.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,21.0,0.0,0.0
Executing,0.0,0.0,0.0,2.0,50.0,0.0,0.0
Released,65.0,4.0,1.0,25.0,25.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,21.0,0.0,0.0
1,Executing,0.0,0.0,0.0,2.0,50.0,0.0,0.0
2,Released,65.0,4.0,1.0,25.0,25.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,21.0,0.0,0.0
1,Executing,0.0,0.0,0.0,2.0,50.0,0.0,0.0
2,Released,65.0,4.0,1.0,25.0,25.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()